In [1]:
from subpred.protein_go_datasets import get_transmembrane_transporter_dataset, get_stats

In [2]:
dataset_yeast = get_transmembrane_transporter_dataset(  # yeast
    organism_ids={559292},
    swissprot_only=True,
    datasets_path="../data/datasets/",
    exclude_iea_go_terms=False,
    max_sequence_evidence_code=1,
    remove_proteins_without_gene_names=True,
)
get_stats(*dataset_yeast)

cd-hit: clustered 347 sequences into 291 clusters at threshold 50
cd-hit: clustered 347 sequences into 318 clusters at threshold 70
cd-hit: clustered 347 sequences into 339 clusters at threshold 90
cd-hit: clustered 347 sequences into 347 clusters at threshold 100


n_transporters  \
swissprot_reviewed has_gene_name go_evidence   protein_existence_evidence clustering                   
True               True          computational protein_level              50                     222   
                                                                          70                     246   
                                                                          90                     266   
                                                                          100                    274   
                                                                          None                   274   
                                 experiment    protein_level              50                     263   
                                                                          70                     287   
                                                                          90                     308   
                                                                          100                    316   
                                                                          None                   316   

                                                                                      n_terms  
swissprot_reviewed has_gene_name go_evidence   protein_existence_evidence clustering           
True               True          computational protein_level              50              130  
                                                                          70              131  
                                                                          90              131  
                                                                          100             131  
                                                                          None            131  
                                 experiment    protein_level              50              280  
                                                                          70              282  
                                                                          90              283  
                                                                          100             283  
                                                                          None            283

In [3]:
dataset_meta = get_transmembrane_transporter_dataset(  # meta
    organism_ids=[3702, 9606, 83333, 559292],
    swissprot_only=True,
    datasets_path="../data/datasets/",
    exclude_iea_go_terms=True,
    max_sequence_evidence_code=1,
    remove_proteins_without_gene_names=True,
)
get_stats(*dataset_meta)

cd-hit: clustered 2260 sequences into 1741 clusters at threshold 50
cd-hit: clustered 2260 sequences into 2060 clusters at threshold 70
cd-hit: clustered 2260 sequences into 2224 clusters at threshold 90
cd-hit: clustered 2260 sequences into 2256 clusters at threshold 100


n_transporters  \
swissprot_reviewed has_gene_name go_evidence protein_existence_evidence clustering                   
True               True          experiment  protein_level              50                    1741   
                                                                        70                    2060   
                                                                        90                    2224   
                                                                        100                   2256   
                                                                        None                  2260   

                                                                                    n_terms  
swissprot_reviewed has_gene_name go_evidence protein_existence_evidence clustering           
True               True          experiment  protein_level              50              718  
                                                                        70              740  
                                                                        90              751  
                                                                        100             751  
                                                                        None            751

In [4]:
dataset_athaliana = get_transmembrane_transporter_dataset(  # athaliana
    organism_ids=[3702],
    swissprot_only=True,
    datasets_path="../data/datasets/",
    exclude_iea_go_terms=True,
    max_sequence_evidence_code=1,
    remove_proteins_without_gene_names=True,
)
get_stats(*dataset_athaliana)

cd-hit: clustered 420 sequences into 276 clusters at threshold 50
cd-hit: clustered 420 sequences into 344 clusters at threshold 70
cd-hit: clustered 420 sequences into 403 clusters at threshold 90
cd-hit: clustered 420 sequences into 419 clusters at threshold 100


n_transporters  \
swissprot_reviewed has_gene_name go_evidence protein_existence_evidence clustering                   
True               True          experiment  protein_level              50                     276   
                                                                        70                     344   
                                                                        90                     403   
                                                                        100                    419   
                                                                        None                   420   

                                                                                    n_terms  
swissprot_reviewed has_gene_name go_evidence protein_existence_evidence clustering           
True               True          experiment  protein_level              50              264  
                                                                        70              281  
                                                                        90              288  
                                                                        100             288  
                                                                        None            288

In [5]:
dataset_human = get_transmembrane_transporter_dataset(  # human
    organism_ids=[9606],
    swissprot_only=True,
    datasets_path="../data/datasets/",
    exclude_iea_go_terms=True,
    max_sequence_evidence_code=1,
    remove_proteins_without_gene_names=True,
)
get_stats(*dataset_human)

cd-hit: clustered 1121 sequences into 846 clusters at threshold 50
cd-hit: clustered 1121 sequences into 1039 clusters at threshold 70
cd-hit: clustered 1121 sequences into 1110 clusters at threshold 90
cd-hit: clustered 1121 sequences into 1118 clusters at threshold 100


n_transporters  \
swissprot_reviewed has_gene_name go_evidence protein_existence_evidence clustering                   
True               True          experiment  protein_level              50                     846   
                                                                        70                    1039   
                                                                        90                    1110   
                                                                        100                   1118   
                                                                        None                  1121   

                                                                                    n_terms  
swissprot_reviewed has_gene_name go_evidence protein_existence_evidence clustering           
True               True          experiment  protein_level              50              534  
                                                                        70              563  
                                                                        90              570  
                                                                        100             570  
                                                                        None            570

In [6]:
dataset_ecoli = get_transmembrane_transporter_dataset(  # ecoli
    organism_ids=[83333],
    swissprot_only=True,
    datasets_path="../data/datasets/",
    exclude_iea_go_terms=True,
    max_sequence_evidence_code=1,
    remove_proteins_without_gene_names=True,
)
get_stats(*dataset_ecoli)

cd-hit: clustered 403 sequences into 372 clusters at threshold 50
cd-hit: clustered 403 sequences into 395 clusters at threshold 70
cd-hit: clustered 403 sequences into 403 clusters at threshold 90
cd-hit: clustered 403 sequences into 403 clusters at threshold 100


n_transporters  \
swissprot_reviewed has_gene_name go_evidence protein_existence_evidence clustering                   
True               True          experiment  protein_level              50                     372   
                                                                        70                     395   
                                                                        90                     403   
                                                                        100                    403   
                                                                        None                   403   

                                                                                    n_terms  
swissprot_reviewed has_gene_name go_evidence protein_existence_evidence clustering           
True               True          experiment  protein_level              50              355  
                                                                        70              359  
                                                                        90              360  
                                                                        100             360  
                                                                        None            360

## Clustering

## Sequence Features

In [7]:
# features standardized
# features non-standardized


from subpred.features import calculate_features

testdf = dataset_athaliana[0].head(20)
display(testdf)

df_features_std_at = calculate_features(testdf.sequence, standardize_samples=True)
df_features_at = calculate_features(testdf.sequence, standardize_samples=False)

df_features_at

,sequence,reviewed,protein_existence,organism_id,protein_names
Uniprot,,,,,
F4IHS9,MGEMKSMQMGVIGALFLSVASSVSIVICNKALMTNLGFPFATTLTS...,True,1,3702,UDP-xylose transporter 1
F4KFS7,MEKSNQPVHVTLSELKDGDKEIVDAEFLVDLLESYRFGKDNVPARE...,True,1,3702,Membrane protein of ER body 2
O49447,MDGSKHPSVFQKLHGQSYLINRLSPSVQARGYCVSGAYVNGGLQSL...,True,1,3702,"ADP,ATP carrier protein 3, mitochondrial (ADP/..."
O65272,MDFASSVQRQSMFHGGADFASYCLPNRMISAKLCPKGLGGTRFWDP...,True,1,3702,"K(+) efflux antiporter 2, chloroplastic (AtKEA2)"
O80725,MASESGLNGDPNILEEVSETKRDKEEEEEVKKTEKKDEEHEKTKTV...,True,1,3702,ABC transporter B family member 4 (ABC transpo...
P18064,MGLLCSRSRHHTEDTDENTQAAEIERRIEQEAKAEKHIRKLLLLGA...,True,1,3702,Guanine nucleotide-binding protein alpha-1 sub...
P46032,MGSIEEEARPLIEEGLILQEVKLYAEDGSVDFNGNPPLKEKTGNWK...,True,1,3702,Protein NRT1/ PTR FAMILY 8.3 (AtNPF8.3) (Histi...
P61837,MEGKEEDVRVGANKFPERQPIGTSAQSDKDYKEPPPAPFFEPGELS...,True,1,3702,Aquaporin PIP1-1 (AtPIP1;1) (Plasma membrane a...
Q08733,MEGKEEDVRVGANKFPERQPIGTSAQTDKDYKEPPPAPFFEPGELS...,True,1,3702,Aquaporin PIP1-3 (AtPIP1;3) (Plasma membrane i...


,AAC__A,AAC__C,AAC__D,AAC__E,AAC__F,AAC__G,AAC__H,AAC__I,AAC__K,AAC__L,...,PSSM_90_3__VL,PSSM_90_3__VK,PSSM_90_3__VM,PSSM_90_3__VF,PSSM_90_3__VP,PSSM_90_3__VS,PSSM_90_3__VT,PSSM_90_3__VW,PSSM_90_3__VY,PSSM_90_3__VV
F4IHS9,0.055556,0.017544,0.023392,0.017544,0.076023,0.064327,0.023392,0.067251,0.055556,0.154971,...,0.613365,0.312649,0.591885,0.489260,0.295943,0.353222,0.489260,0.281623,0.357995,0.852029
F4KFS7,0.041818,0.020000,0.052727,0.118182,0.029091,0.058182,0.005455,0.060000,0.060000,0.092727,...,0.525714,0.408571,0.531429,0.502857,0.422857,0.522857,0.560000,0.277143,0.380000,0.762857
O49447,0.100264,0.007916,0.042216,0.021108,0.052770,0.126649,0.013193,0.047493,0.073879,0.102902,...,0.453704,0.245370,0.465278,0.391204,0.256944,0.321759,0.384259,0.261574,0.331019,0.664352
O65272,0.106474,0.007666,0.042589,0.077513,0.040886,0.075809,0.010221,0.057922,0.049404,0.126917,...,0.516928,0.311313,0.498761,0.380677,0.312139,0.392238,0.449216,0.246078,0.307184,0.763006
O80725,0.097978,0.006221,0.053655,0.056765,0.052100,0.085537,0.010109,0.076205,0.063764,0.094090,...,0.457353,0.204412,0.472794,0.325735,0.166912,0.275000,0.386029,0.138235,0.264706,0.875000
P18064,0.052219,0.015666,0.057441,0.107050,0.052219,0.052219,0.018277,0.057441,0.091384,0.101828,...,0.514286,0.353247,0.480519,0.379221,0.293506,0.368831,0.477922,0.272727,0.353247,0.737662
P46032,0.094017,0.022222,0.037607,0.046154,0.066667,0.083761,0.006838,0.073504,0.037607,0.099145,...,0.519862,0.265976,0.500864,0.416235,0.240069,0.307427,0.414508,0.240069,0.352332,0.804836
P61837,0.118881,0.013986,0.027972,0.031469,0.066434,0.111888,0.024476,0.080420,0.041958,0.073427,...,0.414286,0.180000,0.380000,0.302857,0.157143,0.237143,0.302857,0.177143,0.280000,0.731429
Q08733,0.122378,0.013986,0.027972,0.031469,0.069930,0.111888,0.024476,0.083916,0.038462,0.073427,...,0.430199,0.190883,0.393162,0.316239,0.182336,0.259259,0.313390,0.190883,0.279202,0.735043
Q0WT48,0.071325,0.014556,0.050946,0.094614,0.034934,0.062591,0.013100,0.053857,0.077147,0.101892,...,0.574144,0.328897,0.530418,0.422053,0.231939,0.378327,0.442966,0.228137,0.347909,0.800380


## NLP features

In [8]:
datasets = [
    dataset_ecoli,
    dataset_athaliana,
    dataset_yeast,
    dataset_human,
    dataset_meta,
]
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.version.cuda)
print(torch.cuda.get_arch_list())
# warning is okay, the mentioned compute capability is not used and will disappear once RTX5000 support enters the stable branch of torch
# TODO check if new torch build available

True
NVIDIA GeForce RTX 5070 Ti
12.6
['sm_50', 'sm_60', 'sm_61', 'sm_70', 'sm_75', 'sm_80', 'sm_86', 'sm_89', 'sm_90', 'compute_90']


/home/andy/miniforge3/envs/subpred_deeplearning/lib/python3.12/site-packages/torch/cuda/__init__.py:235: UserWarning: 
NVIDIA GeForce RTX 5070 Ti with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_50 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 sm_89 sm_90 compute_90.
If you want to use the NVIDIA GeForce RTX 5070 Ti GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(


## ProtT5

In [9]:
sequences = dataset_athaliana[0].iloc[:20].sequence
accessions = dataset_athaliana[0].iloc[:20].index.to_numpy().ravel()

In [10]:
from transformers import T5Tokenizer, T5EncoderModel
import torch
import re
import pandas as pd
import numpy as np


def get_protT5_embeddings(sequences: pd.Series, batch_size: int = 1):
    assert batch_size > 0 and batch_size <= len(sequences)
    assert not sequences.str.contains("[UZOB]", regex=True).any()

    n_chunks = len(sequences) / batch_size

    sequences_chunk_idxs = np.array_split(
        np.arange(sequences.shape[0]), indices_or_sections=n_chunks
    )
    sequences_chunks = [
        sequences.iloc[sequences_chunk_idx]
        for sequences_chunk_idx in sequences_chunk_idxs
    ]

    print("loading model...")
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Load the tokenizer
    tokenizer = T5Tokenizer.from_pretrained(
        "Rostlab/prot_t5_xl_half_uniref50-enc", do_lower_case=False
    )

    # Load the model
    model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc").to(
        device
    )

    # slower, cpu does not support half precision
    if device == torch.device("cpu"):
        model.to(torch.float32)

    print("generating embeddings...")

    for sequences_chunk in sequences_chunks:
        chunk_accessions_list = sequences_chunk.index.to_list()
        chunk_sequences_list = sequences_chunk.to_list()
        chunk_sequences_list_encoded = [
            " ".join(list(seq)) for seq in chunk_sequences_list
        ]
        ids = tokenizer(
            chunk_sequences_list_encoded, add_special_tokens=True, padding="longest"
        )
        input_ids = torch.tensor(ids["input_ids"]).to(device)
        attention_mask = torch.tensor(ids["attention_mask"]).to(device)
        # generate embeddings
        with torch.no_grad():
            embedding_repr = model(input_ids=input_ids, attention_mask=attention_mask)
        embeddings_batch = list()
        # extract residue embeddings for the first ([0,:]) sequence in the batch and remove padded & special tokens ([0,:7])
        for pos, sequence in enumerate(chunk_sequences_list):
            emb = embedding_repr.last_hidden_state[
                pos, : len(sequence)
            ]  # shape (7 x 1024)
            emb_per_protein = emb.mean(dim=0).cpu().numpy()  # shape (1024)
            embeddings_batch.append(emb_per_protein)
        torch.cuda.empty_cache()

        yield chunk_accessions_list, embeddings_batch

2025-05-18 19:26:56.061169: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-18 19:26:56.083948: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747589216.096890   84933 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747589216.101230   84933 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-18 19:26:56.127649: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## Alphafold download and mini3di encoding

In [11]:
# import os
# import requests
# import urllib
# import gzip
# import shutil

# # TODO confidence scores of af prediction?
# test_df = dataset_athaliana[0].iloc[:20]
# accession = test_df.index[3]
# accession

# af_dir = "../data/datasets/pdb/"
# af_string = f"AF-{accession}-F1-model_v4.pdb"
# af_path = af_dir + af_string
# af_path_gz = af_path + ".gz"
# af_pdb_url = f"https://alphafold.ebi.ac.uk/files/AF-{accession}-F1-model_v4.pdb"

# # TODO check if experiment PDB file exists?
# if not os.path.exists(af_path):
#     print(f"PDB for {accession} not found in folder {af_dir}")
#     if os.path.exists(af_path_gz):
#         print("Found gzipped PDB, extracting...")
#         # subprocess.run(["gunzip", af_path_gz])
#         with gzip.open(af_path_gz, 'rb') as file_pdb_gz:
#             with open(af_path, 'wb') as file_pdb:
#                 shutil.copyfileobj(file_pdb_gz, file_pdb)
#     elif requests.get(af_pdb_url).status_code == 200:
#         print("Found missing PDB online, downloading...")
#         urllib.request.urlretrieve(af_pdb_url, af_path)
#     else:
#         print(f"PDB for {accession} not found online or in archive, skipping")
# if os.path.exists(af_path):
#     pass

# def get_pdb(accession:str):
# import mini3di
# from Bio.PDB import PDBParser

# parser = PDBParser(QUIET=True)
# struct = parser.get_structure(accession, af_path)

# encoder = mini3di.Encoder()

# for chain in struct.get_chains():
#     states = encoder.encode_chain(chain)
#     sequence_3di = encoder.build_sequence(states)
#     print(chain.get_id(), sequence_3di)

# TODO chains in alphabetical order, merge
# TODO lowercase

## Foldseek encoding

manual in readme: https://github.com/mheinzinger/ProstT5

```
foldseek createdb directory_with_PDBs queryDB
foldseek lndb queryDB_h queryDB_ss_h
foldseek convert2fasta queryDB_ss queryDB_ss.fasta
```

TODO do everything in raw and preprocessing, then use mini3di as backup in case some pdbs are missing?

Dynamic downloader for missing PDBs:

## ProstT5

In [12]:
from transformers import T5Tokenizer, T5EncoderModel
import torch


def get_prostT5_embeddings(sequences: pd.Series, batch_size: int = 1):
    # upper case: aa. lower case: 3Di
    # sequences = sequences_3Di_test
    # batch_size = 1

    n_chunks = len(sequences) / batch_size

    sequences_chunk_idxs = np.array_split(
        np.arange(sequences.shape[0]), indices_or_sections=n_chunks
    )
    sequences_chunks = [
        sequences.iloc[sequences_chunk_idx]
        for sequences_chunk_idx in sequences_chunk_idxs
    ]

    print("loading model...")
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device.type)
    # Load the tokenizer
    tokenizer = T5Tokenizer.from_pretrained("Rostlab/ProstT5_fp16", do_lower_case=False)
    # Load the model
    model = T5EncoderModel.from_pretrained("Rostlab/ProstT5_fp16").to(device)
    # only GPUs support half-precision currently; if you want to run on CPU use full-precision (not recommended, much slower)
    model.float() if device.type == "cpu" else model.half()
    # model = model.eval()
    print("generating embeddings")

    for pos, sequences_chunk in enumerate(sequences_chunks):
        # print(f"generating embeddings for batch {pos+1} out of {len(sequences_chunks)}")
        chunk_accessions_list = sequences_chunk.index.to_list()
        chunk_sequences_list = sequences_chunk.to_list()
        chunk_sequences_list_encoded = [" ".join(seq) for seq in chunk_sequences_list]
        # if you go from AAs to 3Di (or if you want to embed AAs), you need to prepend "<AA2fold>"
        # if you go from 3Di to AAs (or if you want to embed 3Di), you need to prepend "<fold2AA>"
        chunk_sequences_list_encoded = [
            (
                "<AA2fold>" + " " + s if s.isupper() else "<fold2AA>" + " " + s
            )  # this expects 3Di sequences to be already lower-case
            for s in chunk_sequences_list_encoded
        ]
        # tokenize sequences and pad up to the longest sequence in the batch
        ids = tokenizer.batch_encode_plus(
            chunk_sequences_list_encoded,
            add_special_tokens=True,
            padding="longest",
            return_tensors="pt",
        ).to(device)
        # generate embeddings
        with torch.no_grad():
            embedding_repr = model(ids.input_ids, attention_mask=ids.attention_mask)

        embeddings_3D = list()
        # extract residue embeddings for the first ([0,:]) sequence in the batch and remove padded & special tokens, incl. prefix ([0,1:8])
        for pos, sequence_3Di in enumerate(chunk_sequences_list):
            emb = embedding_repr.last_hidden_state[pos, 1 : len(sequence_3Di) + 1]

            # if you want to derive a single representation (per-protein embedding) for the whole protein
            emb_per_protein = emb.mean(dim=0).cpu().numpy()  # shape (1024)
            embeddings_3D.append(emb_per_protein)

        torch.cuda.empty_cache()

        yield chunk_accessions_list, embeddings_3D

In [13]:


# torch.cuda.empty_cache()
# # res_gen = get_prostT5_embeddings(sequences=sequences_3Di_test, batch_size=1)
# torch.cuda.empty_cache()

# res = list(res_gen)

In [14]:
import pandas as pd
import numpy as np
from pathlib import Path


def get_nlp_features(
    sequences: pd.Series,
    batch_size: int = 1,
    model="protT5",
    sequence_type="AA",
    cache_folder: Path = Path("../data/datasets/embeddings/"),
):
    # sequences: index uniprot accession, value sequence, lowercase means 3Di, uppercase means amino acid, only 20 AAs allowed
    # model: one of protT5, prostT5, TODO more with bioencodings package?
    # sequence_type: AA or 3Di
    assert cache_folder.exists() and cache_folder.is_dir()

    embedding_file_paths = sequences.index.to_series().map(
        lambda accession: cache_folder / f"{accession}_{model}_{sequence_type}.npy"
    )

    sequences_noembeddings = sequences[
        ~embedding_file_paths.map(lambda p: p.exists())
    ]

    if len(sequences_noembeddings) > 0:
        match model:
            case "protT5":
                embeddings_res = get_protT5_embeddings(
                    sequences=sequences_noembeddings, batch_size=batch_size
                )
            case "prostT5":
                embeddings_res = get_prostT5_embeddings(
                    sequences=sequences_noembeddings, batch_size=batch_size
                )
                # TODO save
            case other:
                raise ValueError(f"Model not known: {other}")

        # save to cache
        for chunk_accessions, chunk_embeddings in embeddings_res:
            for chunk_accession, chunk_embedding in zip(
                chunk_accessions, chunk_embeddings
            ):
                np.save(embedding_file_paths[chunk_accession], chunk_embedding)

        del embeddings_res

    torch.cuda.empty_cache()
    embeddings = np.array([np.load(x) for x in embedding_file_paths.values])
    return pd.DataFrame(data=embeddings, index=embedding_file_paths.index)  # TODO feature names


In [15]:
torch.cuda.empty_cache()

In [41]:
dataset_meta = get_transmembrane_transporter_dataset(  # meta
    organism_ids=[3702, 9606, 83333, 559292],
    swissprot_only=False,
    datasets_path="../data/datasets/",
    exclude_iea_go_terms=False,
    max_sequence_evidence_code=2,
    remove_proteins_without_gene_names=False,
)
get_stats(*dataset_meta)

cd-hit: clustered 6512 sequences into 2417 clusters at threshold 50
cd-hit: clustered 6512 sequences into 3088 clusters at threshold 70
cd-hit: clustered 6512 sequences into 3832 clusters at threshold 90
cd-hit: clustered 6512 sequences into 5827 clusters at threshold 100


n_transporters  \
swissprot_reviewed has_gene_name go_evidence   protein_existence_evidence clustering                   
False              False         computational protein_level              50                       1   
                                                                          70                       1   
                                                                          90                       5   
                                                                          100                     12   
                                                                          None                    13   
                                               transcript_level           50                      60   
                                                                          70                      82   
                                                                          90                     159   
                                                                          100                    702   
                                                                          None                   820   
                                 experiment    transcript_level           50                       1   
                                                                          70                       1   
                                                                          90                       1   
                                                                          100                      2   
                                                                          None                     2   
                   True          computational protein_level              50                     232   
                                                                          70                     324   
                                                                          90                     572   
                                                                          100                   1360   
                                                                          None                  1608   
                                               transcript_level           50                      38   
                                                                          70                      59   
                                                                          90                     119   
                                                                          100                    585   
                                                                          None                   851   
                                 experiment    protein_level              50                      25   
                                                                          70                      33   
                                                                          90                      38   
                                                                          100                     41   
                                                                          None                    42   
                                               transcript_level           50                       3   
                                                                          70                       6   
                                                                          90                       7   
                                                                          100                     10   
                                                                          None                    11   
True               True          computational protein_level              50                    1310   
                                                                          70                    1651   
                              

In [42]:
all_sequences = dataset_meta[0].sequence
# TODO go back to ProstT5 regular?

In [43]:
from subpred.util import load_data

sequences_3Di_all = load_data("3Di_alphafold4")

# TODO make this faster: save model somehow after loading?

all_sequences_3Di = sequences_3Di_all[sequences_3Di_all.index.isin(all_sequences.index)].sequence3Di
all_sequences_3Di

Uniprot
P21817    dpppvvvvvlwfawaqfkwwkwfwdqdpndtwiwtwfflqfltffi...
P98161    ddddddddddddddpddddddddddddldfddwdppwdwdqdppsa...
Q01484    ddddddddddddddddddddddddddpppvllvvllvcllvlvqvv...
Q12955    dvvvvvvvvvvvvpppdddddppvvvvvvvvvvvvvvvvvlqvvll...
Q14571    dddpdfdfdaqffkwfkfkddpatatwfdlfllfffiwgqrvqddq...
                                ...                        
Q9ZVK6    dvvvvvvvvvvvcvvvvvvvvvvvlafadlvllllllllllllllv...
Q9ZVX8    ddddpdppdppdpdqdfddfadqddpvlvvdpllvqllvlllqlll...
Q9ZWG1    dppppppppddpvlllvlllvlllvllqvlqllvllllqllqqfpf...
Q9ZWT3    dvvvclqapqsvlqpfdddpllvvllllllllllllllqlqllllc...
Q9ZZX1    dcccaqvnlalqsqlvllqvqqvvllvvlvvlvvvlvqqvvdpdgd...
Name: sequence3Di, Length: 3251, dtype: object

In [44]:
set(all_sequences_3Di.index) - set(all_sequences.index)

set()

In [45]:
set(all_sequences.index) - set(all_sequences_3Di.index)

{'A0A077',
 'Q0WLU6',
 'E9PRM7',
 'E5RFP6',
 'A0A7P0T9Z9',
 'A0A0J9YWM3',
 'E9PGB2',
 'B4DY10',
 'B4DWQ9',
 'B1AKY9',
 'A0A1S6GNA3',
 'Q86V90',
 'A0A9E8M488',
 'A0A1P8AZ62',
 'B9DFT4',
 'A0A1L1Z877',
 'X5DNL6',
 'B4E2Q0',
 'M4PJG5',
 'A0A1P8B6V7',
 'B4DGM6',
 'Q8NBT6',
 'B3KVV3',
 'J3QRB3',
 'A0A1B0GWE1',
 'H0Y860',
 'B3KY44',
 'A0A1W2PPS4',
 'B4DGC4',
 'B4DXX8',
 'A0A286YF72',
 'B4DSF6',
 'B2RMP2',
 'Q13717',
 'Q9UPC8',
 'Q5W1L6',
 'A0A8V8TN05',
 'J3QTB0',
 'Q53F03',
 'B4DME5',
 'A0A1P8B3D9',
 'A0A8V8TM76',
 'A0A1P8B939',
 'A0A3S6CTX0',
 'A0A2R8Y6X9',
 'E9PNE7',
 'A0A8I5KSQ4',
 'A7E249',
 'A0A7P0T8H0',
 'Q8NC07',
 'Q658V6',
 'B4DGA8',
 'A0A1B0GUE5',
 'Q96AI9',
 'D6R9N3',
 'Q59G51',
 'A0ME30',
 'F4IIZ3',
 'A0A384N6F4',
 'A2VBC8',
 'A0A1P8AYR6',
 'A7VJS0',
 'F8VVM2',
 'A0A804HJH4',
 'F5H6S9',
 'B4DR53',
 'A0A1W2PRY7',
 'Q14C71',
 'A0A1I9LP14',
 'Q9UPF3',
 'A0A0C4DGE9',
 'A0AAQ5BII6',
 'B2ZGP8',
 'Q59GU3',
 'A0A1P8AN90',
 'B2R9H8',
 'A0A8I5KY05',
 'B4DPI4',
 'B9DGX9',
 'Q14CR0',
 'Q8TF11

In [46]:
sequences_embeddings_protT5_AA = get_nlp_features(all_sequences, model="protT5", sequence_type="AA")
sequences_embeddings_prostT5_AA = get_nlp_features(all_sequences, model="prostT5", sequence_type="AA")
sequences_embeddings_prostT5_3Di = get_nlp_features(all_sequences_3Di, model="prostT5", sequence_type="3Di")

loading model...
generating embeddings...
loading model...
cuda
generating embeddings
